# Ground truth recordings for validation of spike sorting algorithms

@author : Samuel Garcia


Giulia  Spampinato from Institut de la Vision, Paris, have publish data from mice retina recorded invitro with from dense array.
The data contain one ground truth to benchmark spike sorting tools. The ground truth is recorded with juxta celullar pipet.

Here the official publication of this open dataset:
https://zenodo.org/record/1205233#.W9mq1HWLTIF


This datasets was used by Pierre Yger publish spyking circus:
https://elifesciences.org/articles/34518

After inspecting the juxta cellular, some recordings of the juxta cellular itself have no a good enough quality to be a "ground truth". To be a "ground truth" a recording must no doubt about the threhold, peak snr, amplitude stability.

At the end some files have been removed for the main study in **"sorter_comparison.ipynb"**.


Here some detail about that.

First, we have to run the script "detect_ground_truth_spike_on_juxta.py".

This script:
  * unzip evrything
  * run a juxta cellular detection
  * generate figure to check manual juxta cellular quality
  * compute the peak snr on the max channelin the mea.

Before we need:
  * to create a folder **basedir** somwhere.
  * a subfolder **basedir/original_files** that contain all zip downloded (20160415_patch2.tar.gz, ...)

run the script detect_ground_truth_spike_on_juxta.py

After we can:
  * inscpect in each folder some figures.






# Cretirium to keep or remove a file

Having a very string ground truth is crucial because all following metrics for sorter will bias by the fact that ground truth could itself have False Negative or False Positive.


In the script we choose a hight threhold value for peak detection: **thresh = med + 8\*mad**
  * **med** been the median of the signal (the baseline)
  * **mad** been the median absolut deviation a robust std estimation
  * 8 is a quite high relative threshold thta ensure no noise at all.
  
  
Two main criteria to keep a recording:
  * the distribution of the peak value of the juxta celullar must look like a gaussian. Because:
     * troncated gaussian warn about been false negative (miss) in ground truth
     * multi modal guassian warn us about drift in amplitude or two cells




# List of kept recording (9)


```python
'20160415_patch2',
'20170803_patch1',
'20170623_patch1', 
'20170622_patch1',
'20160426_patch3', 
'20170725_patch1',
'20170621_patch1',
'20160426_patch2', 
'20170728_patch2'
```


# List of non kept recording (10)

```python
'20170706_patch2'
'20170713_patch1'
'20170629_patch2'
'20170622_patch2'
'20170726_patch1'
'20170706_patch1'
'20170706_patch3'
'20170627_patch1'
'20170630_patch1'
'20170629_patch3'
```

Some reader could find me too strict, I but I prefer safe final results.
Feel free to modify this list as you want you your own criteria.

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import os

# path
basedir = '/home/samuel/DataSpikeSorting/Pierre/zenodo/'
recording_folder = basedir + 'original_files/'
ground_truth_folder = basedir + 'ground_truth/'

In [12]:
# 2 simple functions

def get_juxta_filename(rec_name):
    # find the juxta file
    dirname = recording_folder + rec_name + '/'
    for f in os.listdir(dirname):
        if  f.endswith('juxta.raw'):
            juxta_filename = dirname + f
            return juxta_filename

def plot_juxta_amplitude(rec_name):
    juxta_filename = get_juxta_filename(rec_name)
    juxta_sig = np.memmap(juxta_filename, dtype='float32')
    
    gt_indexes = gt_folder+'juxta_peak_indexes.raw'
    gt_indexes = np.fromfile(ground_truth_folder + rec_name + '/juxta_peak_indexes.raw', dtype='int64')
    
    fig, ax = plt.subplots()
    count, bins = np.histogram(juxta_sig[gt_indexes], bins=np.arange(np.min(juxta_sig[gt_indexes]), 0,  0.5))
    ax.plot(bins[:-1], count)
    ax.axvline(-thresh, color='k', ls='--')
    ax.set_title('juxta peak amplitude - ' + rec_name)



## Figure for 20170706_patch2

In [ ]:
plot_juxta_amplitude('20170706_patch2')

## Figure for 20170713_patch1

In [ ]:
plot_juxta_amplitude('20170713_patch1')

## Figure for 20170629_patch2

In [ ]:
plot_juxta_amplitude('20170629_patch2')

## Figure for 20170622_patch2

In [ ]:
plot_juxta_amplitude('20170622_patch2')

## Figure for 20170726_patch1

In [ ]:
plot_juxta_amplitude('20170726_patch1')

## Figure for 20170706_patch1

In [ ]:
plot_juxta_amplitude('20170706_patch1')

## Figure for 20170706_patch3

In [ ]:
plot_juxta_amplitude('20170706_patch3')

## Figure for 20170627_patch1

In [ ]:
plot_juxta_amplitude('20170627_patch1')

## Figure for 20170630_patch1

In [ ]:
plot_juxta_amplitude('20170630_patch1')

## Figure for 20170629_patch3

In [ ]:
plot_juxta_amplitude('20170629_patch3')